Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

In [1]:
NAME = 'Paul Natland'
UMICH_UNIQNAME = 'pnatland'

In [2]:
"""
ACKNOWLEDGEMENTS: For this assignment, I benefited from lecture videos, searches on Stacks Overflow, and conversations on SLACK
"""

'\nACKNOWLEDGEMENTS: For this assignment, I benefited from lecture videos, searches on Stacks Overflow, and conversations on SLACK\n'

---

In [3]:
version = "v2.3.060120"

# SIADS 515 Week 2 Homework (HW2)

For this assignment you'll be using a Python class we've written that provides data from the `metrics.csv` and `position.csv` files in the `assets` directory. You will not need to directly access those files to complete the assingment, but you may want to see their contents.

In [4]:
from assets.api import BikeRideApi
api = BikeRideApi('assets/metrics.csv', 'assets/position.csv')

This class provides a `get_data()` method.

You can use the built-in `help()` function to read the docstring for `get_data()` ...

In [5]:
help(api.get_data)

Help on method get_data in module assets.api:

get_data(dataset_name, offset, count=5) method of assets.api.BikeRideApi instance
    Returns a tuple of "count" tuples starting from "offset" for the
    given "dataset"
    
    Arguments:
        dataset_name {str} -- The dataset to pull data from ("metrics"
            or "position")
        offset {int} -- Offset from the beginning of the dataset (must
            be greater than 0)
    
    Keyword Arguments:
        count {int} -- Number of rows to include (default: 5; must be 
            between 1 and 10 inclusive)
    
    Raises:
        ValueError: Invalid parameter values
    
    Returns:
        tuple -- A tuple of "count" tuples starting from "offset" for the
            given "dataset". Returns an empty tuple when there is no more 
            data.



__Exploration__

In [6]:
api.get_data('metrics', offset=0)

((10.87, 285.79999999999995, 1.773),
 (71.85, 285.0, 5.533),
 (108.02, 284.0, 6.485),
 (170.23, 284.0, 6.951),
 (229.27, 285.0, 6.224))

In [7]:
api.get_data('position', offset=0)

((504719750, -998493490),
 (504717676, -998501870),
 (504716354, -998506792),
 (504714055, -998515244),
 (504711900, -998523278))

In [8]:
api.get_data('position', offset=0)[2][0]

504716354

In [9]:
api.get_data('metrics', offset=4, count=1)

((229.27, 285.0, 6.224),)

In [10]:
#not desired format
#list(zip(api.get_data('metrics', offset=0), api.get_data('position', offset=0)))

In [11]:
n = 0
while True:
    try:
        api.get_data('position', offset=n)[0]
        n += 1
    except IndexError:
        break
print(n)

207


In [12]:
#this is the desired format in tuple form
api.get_data('metrics', offset=0)[0] + api.get_data('position', offset=0)[0]

(10.87, 285.79999999999995, 1.773, 504719750, -998493490)

---
**NOTE** 

The `get_data()` method artificially adds a delay of 1 millisecond to each call. This mimics the latency that is common when fetching data from network sources like an HTTP API. In the real world, 1 millisecond would be very low latency for an HTTP API, but it adds enough of a delay that you'll be able to see very noticable differences depending on how you optimize your code for efficency.

---
## Question 1
Write a **generator** called `combine_data()` that meets these requirements:
  - It doesn't take any parameters.
  - It must use the `get_data()` method descibed above.
  - It must use a value for the "count" parameter greater than 1.
  - It must use only the Python standard library (e.g. no IPython magic commands and no external libraries like Pandas)
  - It must yield a combined structure for each row in the "metrics" and "position" datasets.
  - Each yielded item must be an iterable (e.g. a list or a tuple is fine), that contains, in this order, the values for:
    - "Distance"
    - "Altitude"
    - "Speed"
    - "Latitude"
    - "Longitude"
  - It must stop yielding when either dataset runs out of data.
  - It must handle cases other than the data in the `assets` directory (e.g. don't hard code dataset lengths, or other values).
  
Note:
  - The `zip` built-in function could be useful in your solution, but is not required.

In [13]:
def combine_data():
    num = 0
    while True:
        try:
            result = api.get_data('metrics', offset=num, count = 2)[0] + api.get_data('position', offset=num, count = 2)[0]
            num += 1
        except IndexError:
            break
        yield result

In [14]:
%%timeit
gen = combine_data()

210 ns ± 2.01 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
%%timeit
gen = combine_data()

list(gen)

477 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# Question 1, Check 1
# 1 point

gen = combine_data()

# Make sure your solution is a generator
from types import GeneratorType
assert type(gen) == GeneratorType, 'wrong type, should be a generator'

# There are no hidden tests in this cell.

In [17]:
# Question 1, Check 2 
# 1 point

gen = combine_data()

# Make sure you stop yielding when one dataset ends 
assert len(list(gen)) == 207, 'wrong number of results'

# There are no hidden tests in this cell.

In [18]:
# Question 1, Check 3
# 1 point.

gen = combine_data()

# Make sure your solution yields the correct 1st row
assert next(gen) == (10.87, 285.79999999999995, 1.773, 504719750, -998493490), 'incorrect data'

# Make sure your solution yields the correct 2nd row
assert next(gen) == (71.85, 285.0, 5.533, 504717676, -998501870), 'incorrect data'

# Make sure your solution yields the correct 3rd row
assert next(gen) == (108.02, 284.0, 6.485, 504716354, -998506792), 'incorrect data'

# There are no hidden tests in this cell.

In [19]:
# Question 1, Check 4
# 1 point

# There are hidden tests in this cell to check that get_data() is called
# NOTE: To get this check to pass, your solution just needs to call get_data()

In [20]:
# Question 1, Check 5
# 1 point.

# NOTE: To get this test to pass, your solution needs to either not specify the "count"
# param, or specifiy a value of greater than 1 for the "count" param.

# There are hidden tests in this cell to check that get_data() is called with "count" > 1

In [21]:
# Question 1, Check 6
# 5 points

# There are hidden tests in this cell to check that at least 90% rows are correct for the given data.

In [22]:
# Question 1, Check 7
# 5 points

# There are hidden tests in this cell to check that at 100% rows are correct for the given data.

In [23]:
# Question 1, Check 8
# 5 points

# There are hidden tests in this cell to check the solution with other input cases.

---
## Question 2
Write a **function** called `get_speed()` that meets these requirements:
  - It must take 1 parameter: an integer that represents the row of data from which to return the speed.
  - It must use the `get_data()` method descibed above.
  - It must use only the Python standard library (e.g. no IPython magic commands and no external libraries like Pandas)
  - It must be decorated with the `lru_cache` decorator from the `functool` package in the standard library.
  - It must return an numeric value for the speed at the given data index.
  - It must return `None` if the data at the given index is empty (past the end of the dataset)
  - It must handle cases other than the data in the `assets` directory (e.g. don't hard code dataset lengths, or other values).
  
Note:
  - It is probably simplest to use a value for the "count" parameter of 1 (unlike the requirement in Question 1).

In [24]:
#gets the speed for row = offset
api.get_data('metrics', offset=4, count=1)[0][2]

6.224

### solution

In [25]:
from functools import lru_cache

@lru_cache(16)
def get_speed(i):
    # i equals the row number
    while True:
        try:
            result = api.get_data('metrics', offset=i, count=1)[0][2]
        except IndexError:
            return None
        return result

The `%timeit` magic command is a quick way to judge the speed of a function.

In [26]:
%timeit get_speed(0)

74.3 ns ± 0.912 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


The `lru_cache` decorator actually turns `get_speed` into a callable object so it still works like the original function, but also has some helpful properities. For example the `__wrapped__` property is the oringal function, so we can see how much worse its performance is by calling that with `%timeit` instead.

This call should be significantly slower than the one above. Given that the underlying method is adding a 1 millisecond delay, the uncached version of `get_speed()` should take at least 1 millisecond.

In [27]:
# This cell will fail if get_speed() is not decorated with lru_cache

%timeit get_speed.__wrapped__(0)

1.14 ms ± 3.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [28]:
# Question 2, Check 1
# 1 point.

# Make sure your solution uses the lru_cache decorator
from functools import _lru_cache_wrapper
assert type(get_speed) == _lru_cache_wrapper, 'solution must be decorated with the lru_cache decorator'

# There are no hidden tests in this cell.

In [29]:
# Question 2, Check 2
# 1 point

# Make sure your solution returns the correct values for the first 3 indexes
assert get_speed(0) == 1.773, 'wrong data value'
assert get_speed(1) == 5.533, 'wrong data value'
assert get_speed(2) == 6.485, 'wrong data value'

# There are no hidden tests in this cell.

In [30]:
# Question 2, Check 3
# 1 point

# Make sure your solution returns the None for indexes past the range of the dataset
assert get_speed(214) == None, 'must return None for indexes past the range of the dataset'
assert get_speed(1000) == None, 'must return None for indexes past the range of the dataset'

# There are no hidden tests in this cell.

In [31]:
# Question 2, Check 4
# 4 points

# There are hidden tests in this cell to check the solution returns at least 90% of the correct results.

In [32]:
# Question 2, Check 5
# 4 points

# There are hidden tests in this cell to check the solution returns 100% of the correct results.

In [33]:
# Question 2, Check 6
# 4 points

# There are hidden tests in this cell to check the solution with other input cases.

---
## Question 2b
Write a **generator** called `speed_rolling_average()` that meets these requirements:
  - It must take 1 parameter: an integer that represents the number of rows over which to calculate a rolling average. This must default to 5.
  - It must use the `get_speed()` method you implemented above for Question 2.
  - It must use only the Python standard library (e.g. no IPython magic commands and no external libraries like Pandas)
  - It must yield an numeric value for the average speed over the previous number of rows as defined by the `period` parameter.
  - It must yield `None` if there have been fewer than `period` number of rows so far in the dataset.
  - It must handle cases other than the data in the `assets` directory (e.g. don't hard code dataset lengths, or other values).

### my better solution

In [34]:
def speed_rolling_average(period=5):
    # the parameter value represents the number of rows to consider for the rolling average
    n = 0
    while n < (period-1):
        yield None
        n+= 1
    i = 0
    while True:
        try:
            t = 0
            speeds_to_average = []
            while t < period:
                value = get_speed(i+t)
                speeds_to_average.append(value)
                t += 1
            yield sum(speeds_to_average)/len(speeds_to_average)
            i += 1
        except TypeError:
            break

In [35]:
%%timeit
gen = speed_rolling_average(5)

list(gen)

247 ms ± 929 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### my first solution

In [36]:
def speed_rolling_average(period=5):
    # the parameter value represents the number of rows to consider for the rolling average
    n = 0
    while True:
        try:
            api.get_data('metrics', offset=n)[0]
            n += 1
        except IndexError:
            break
    for i in range(n):
        if i < (period-1):
            yield None
        else:
            break
    for i in range(n):
        try:
            t = 0
            speeds_to_average = []
            while t < period:
                value = get_speed(i+t)
                speeds_to_average.append(value)
                t += 1
            yield sum(speeds_to_average)/len(speeds_to_average)
        except TypeError:
            break

In [37]:
%%timeit
gen = speed_rolling_average(5)

list(gen)

494 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
gen = speed_rolling_average(5)

len(list(gen))

214

In [39]:
# Question 2b, Check 1
# 1 point.

gen = speed_rolling_average(5)

# Make sure your solution is a generator
from types import GeneratorType
assert type(gen) == GeneratorType, 'wrong type, should be a generator'

# There are no hidden tests in this cell.

In [40]:
# Question 2b, Check 2 
# 1 point

gen = speed_rolling_average(5)

# Make sure you stop yielding when one dataset ends 
assert len(list(gen)) == 214, 'wrong number of results'

# There are no hidden tests in this cell.

In [41]:
# Question 2b, Check 3
# 1 point.

gen = speed_rolling_average(5)

# Make sure your solution yields None for period-1 iterations
assert next(gen) == None, 'should yield None for i < period'
assert next(gen) == None, 'should yield None for i < period'
assert next(gen) == None, 'should yield None for i < period'
assert next(gen) == None, 'should yield None for i < period'

# There are no hidden autograder tests in this cell.

In [42]:
# Question 2b, Check 4
# 1 point.

# Make sure your solution yields the correct first rolling average speed
assert next(gen) == 5.3932, 'wrong data result'

# Make sure your solution yields the correct second rolling average speed
assert next(gen) == 6.1508, 'wrong data result'

# There are no hidden tests in this cell.

In [43]:
# Question 2b, Check 5
# 5 points

# There are hidden tests in this cell to check the solution returns at least 90% of the correct results.

In [44]:
# Question 2b, Check 6
# 5 points

# There are hidden tests in this cell to check the solution returns 100% of the correct results.